In [1]:
import json
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import cv2
import matplotlib.pyplot as plt
import numpy as np
import shutil

# make new coco data set

In [2]:
class DIR:
    src_ann = './../coco/annotations/instances_val2017.json' # path/*.json
    dst_ann = './new_coco/annotations/new_anno.json' # path/*.json
    
    src_img = './../coco/images/val2017/' # image path/ ex)coco/images/val2017/
    dst_img = './new_coco/images/' # image path/ ex)new_coco/images/val2017/

In [3]:
def make_small_coco(DIR, num):
    with open(DIR.src_ann) as f:
        anno = json.load(f)
        
    coco = COCO(DIR.src_ann)
    img_ids = coco.getImgIds()[:num]
    
    new_anno = anno
    new_anno['images'] = coco.loadImgs(img_ids)
    new_anno['annotations'] = coco.loadAnns(coco.getAnnIds(imgIds=img_ids))
    
    # copy images
    for img_anno in new_anno['images']:
        src = DIR.src_img + img_anno['file_name']
        dst = DIR.dst_img + img_anno['file_name']
        shutil.copy(src, dst)
    
    # copy annotation.json
    with open(DIR.dst_ann, 'w') as f:
        json.dump(new_anno, f)
    
    return new_anno

In [4]:
new_anno = make_small_coco(DIR, 100)

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!


# evaluation demo

In [5]:
from tqdm import tqdm

In [6]:
new_anno_json = './new_coco/annotations/new_anno.json'
new_pred_json = './new_coco/annotations/new_pred.json'
pred_json ='./yolov3_predictions.json'

## make new result by image ids

In [7]:
new_anno = COCO(new_anno_json)

with open(pred_json, 'r')as f:
    new_pred = json.load(f)

img_ids = new_anno.getImgIds()
for i in tqdm(range(len(new_pred)-1, -1, -1)):
    if not new_pred[i]['image_id'] in img_ids:
        new_pred.pop(i)

with open(new_pred_json, 'w') as f:
    json.dump(new_pred, f)
    
new_pred = new_anno.loadRes(new_pred_json)  # init predictions api
eval = COCOeval(new_anno, new_pred, 'bbox')

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


100%|██████████| 684416/684416 [00:01<00:00, 362267.79it/s]


Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!


In [8]:
eval.evaluate()
eval.accumulate()
eval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.28s).
Accumulating evaluation results...
DONE (t=0.37s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.506
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.675
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.554
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.573
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.647
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.417
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.633
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.688
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.502
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100